In [9]:
import sys
sys.path.append('g:\\Programming\\YandexShcool\\shared\\')
import imp
import my_imports
imp.reload(my_imports)   # перезагрузка модуля в любом случае
from my_imports import *

In [2]:
newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

In [3]:
X = newsgroups.data
y = newsgroups.target

In [4]:
vectorizer = TfidfVectorizer()
X_scaled = vectorizer.fit_transform(X)

In [11]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(y.size, n_folds=5, shuffle=True, random_state=241)
clf = svm.SVC(kernel='linear', random_state=241)
gs = grid_search.GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(X_scaled, y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=1786, n_folds=5, shuffle=True, random_state=241),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [17]:
for a in gs.grid_scores_:
    quality = a.mean_validation_score # оценка качества по кросс-валидации
    c = a.parameters['C']                # значения параметров
    print('q = %f   c=%f' %(quality, c))

q = 0.552632   c=0.000010
q = 0.552632   c=0.000100
q = 0.552632   c=0.001000
q = 0.552632   c=0.010000
q = 0.950168   c=0.100000
q = 0.993281   c=1.000000
q = 0.993281   c=10.000000
q = 0.993281   c=100.000000
q = 0.993281   c=1000.000000
q = 0.993281   c=10000.000000
q = 0.993281   c=100000.000000


In [18]:
bestC = 1
clf = SVC(random_state=241, C=bestC, kernel='linear')
clf.fit(X_scaled, y)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)

In [86]:
ws = clf.coef_.toarray()[0]
ws = np.abs(ws)
feature_mapping = vectorizer.get_feature_names()
d = zip(ws, feature_mapping)
d.sort()
d.reverse()

In [87]:
i = 0
words = []
while i < 10:
    w = d[i][1]
    words.append(w)
    i += 1
    
words.sort()

In [88]:
answer = ''
for w in words:
    answer += w + ','

answer

u'atheism,atheists,bible,god,keith,moon,religion,sci,sky,space,'

In [73]:
X_train = newsgroups.data
y_train = newsgroups.target

In [74]:
scaler = TfidfVectorizer()
X_train_scaled = scaler.fit_transform(X_train)
# При таком подходе получается, что признаки на обучающем множестве 
# используют информацию из тестовой выборки — но такая ситуация вполне законна, 
# поскольку мы не используем значения целевой переменной из теста. 
X_test_scaled  = scaler.transform(X_train)

In [80]:
X_train_widht  = len(X_train_scaled.toarray()[0])
X_train_height = len(X_train_scaled.toarray())

In [64]:
cs = Series(np.arange(-5, 6))
cs = cs.apply(lambda x: 10**x)
cs

0          0.00001
1          0.00010
2          0.00100
3          0.01000
4          0.10000
5          1.00000
6         10.00000
7        100.00000
8       1000.00000
9      10000.00000
10    100000.00000
dtype: float64

In [86]:
def getMetricByC(inC):
    svc = SVC(random_state=241, C=inC, kernel='linear')
    svc.fit(X_train_scaled, y_train)

    # генератор разбиений, который задает набор 
    # разбиений на обучение и валидацию
    kfold = KFold(X_train_height, n_folds=5,
                  shuffle=True, random_state=241)

    # В качестве метрики используем среднеквадратичную ошибку
    cvsArray = cross_val_score(svc, X=X_train_scaled, y=y_train, cv=kfold)
    cvs = cvsArray.mean()
    return cvs

In [88]:
dictMetrics = {}

for c in cs:
    dictMetrics[c] = getMetricByC(c)

dictMetrics   

{1.0000000000000001e-05: 0.5526360264776301,
 0.0001: 0.5526360264776301,
 0.001: 0.5526360264776301,
 0.01: 0.5526360264776301,
 0.10000000000000001: 0.95016352909879043,
 1.0: 0.99328044066788723,
 10.0: 0.99328044066788723,
 100.0: 0.99328044066788723,
 1000.0: 0.99328044066788723,
 10000.0: 0.99328044066788723,
 100000.0: 0.99328044066788723}

In [89]:
svc = SVC(random_state=241, C=1, kernel='linear')
svc.fit(X_train_scaled, y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)

In [91]:
type(svc.coef_)

scipy.sparse.csr.csr_matrix